---
title: Introduction to Plotting with Python
authors:
  - name: Dustin Wheeler
    email: dustin.wheeler@hunter.cuny.edu
    affiliations:
      - ror: 00g2xk477
      - institution: CUNY – Hunter College
      - department: Chemistry
date: 2024-01-29
numbering:
  heading_2: true
  heading_3: true
---

## Setup

### Library import

We import all the required Python libraries

In [1]:
# File handling
from pathlib import Path
import shutil
import re

# Data manipulation
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("ticks")
sns.despine()

<Figure size 640x480 with 0 Axes>

## Purpose

The goal of this exercise is to familiarize the student with basic data manipulation and plotting using Jupyter notebooks. Many of the concepts are general and can be translated to other software applications. By the end of this exercise, you should be able to

- perform basic manipulations on data columns(formatting, transformations)
- perform simple mathematical operations over a set of data
- plot columns of data in a clear and visually pleasing manner
- perform simple statistical operations on a set of data
- get a line-of-best-fit for multiple datasets
- import data from plain text files (CSV, TSV, DAT, etc.),like those created by many spectrometers and logging instruments


## Introduction

Why do we plot data? Edward Tufte may have said it best:
> At their best, graphics are instruments for reasoning about quantitative information. Often the most effective way to describe, explore, and summarize a set of numbers—even a very large set—is to look at pictures of those numbers.


### A Little Bit About Plots

Plots (also called graphs or charts, though these are not quite the same thing) come in a variety of styles. The can be used to explore data as your are performing analyses or to convey results to a reader as part
of a report. There are a variety of "standard" types which should cover your needs in this course. The main types we will use are

Scatter plot
: Used to show correlation between two variables, each point generally represents an **independent** measurement.
Points are not **connected** by lines, lines are instead used to represent a mathematical model fitting the data.

Line graph
: Indicates continuity in the $x$ dimension, often suggests a single collection session for all data on the "line" (e.g., a UV-Vis or NMR spectrum). Individual points are generally not shown. Lines should never be smoothed (or "splined"), as this treatment no longer accurately represents the data.
: Really, a line graph is just a special case of a scatter plot where the independent variable is continuous.

Histogram
: Represents the distribution of data in a series of "bins", shows the frequency of a measured event. Closely related to density plots.

Here are the important parts of a plot, along with a brief description and guidelines for formatting. 

Title
: Should be descriptive and represent all data in the plot.

Axes
: Should be clearly labeled (category, units, tick numbers). Should have an appropriate range (from slightly less than your minimum value to slightly more than your maximum value).

Fit line
: Should cover the whole spread of the data, but should not predict beyond the data range (you don't know anything about measurements in that region).

Legend
: Should clearly show which set of data is which, trend lines should clearly correlate to a given data set.


## Basic Plotting and Linear Fits

Your first task is to take the data in the file `data.txt` and import it into this notebook.

The folder `raw_data` contains a few datasets in various formats. The file extension doesn't always tell you what type of information is in a file and can sometimes lead you astray. For this exercise, the file `data.txt` contains a set of "tab-delimited" data, meaning that the values in a row are separated by tab characters. Another common format is "comma-delimited" or "comma-separated" data, such as the dataset in the file `anscombe.csv`. You will find that different instruments have different defaults for exporting data, but these two are the most common. Sometimes, a `.dat` extension might be used for a plain-text file to indicate 'data' without indicating the underlying format or delimiter type.

:::{note}
Before proceeding further in this notebook, click on File in the JupyterLab menubar, select "Rename Notebook..."" to give the notebook a title using the following convention:

The title of the notebook should be coherent with file name. Namely, file name should be:    
*author-initials_title.ipynb*    
For example:    
*DW_python_intro.ipynb*

All of your notebooks should follow this convention so readers can tell at a glance what to expect from a given file. 
:::

Before proceeding, make sure you've executed the code cell under the "Library Import" section. "Libraries" are just collections of useful Python code that individuals or groups have created. They generally cover a specific area or hold to a theme. In our notebook template, we have available the following:

[NumPy][np]
: A general purpose numerical library that speeds up number based operations, allows functions to operate on lists of numbers, and gives a large collection of useful mathematical functions (things like polynomial fitting, matrix operations, and common algebraic and trigonometric functions). The core functionality is based around NumPy arrays, very fast n-dimensional arrays of data (a core require- ment for scientific computation).

[SciPy][sp]
: A fundamental library for scientific computing, builds on the NumPy library but includes numeric integration routines, non-linear curve fitting routines, interpolation, numerical optimization, and statistics.

[Pandas][pd]
: A data management library that offers functions for importing, tidying, organizing, and filtering data using the DataFrame construct. Excellent resource for 2-dimensional data (though generally focused on finance and social science applications), utility starts breaking down with more dimensions. Has some good functions for importing and labelling data, but NumPy+Xarray is probably better in almost every case. 

[Xarray][xr]
: A package built on Pandas that introduces labels in the form of dimensions, coordinates, and attributes on top of raw NumPy-like arrays, which allows for more intuitive, more concise, and less error-prone user experience. We will focus on using NumPy and Xarray for most of our data handling.

[Matplotlib][mpl]
: A comprehensive library for creating static, animated, and interactive visualizations in Python. It has become the de-facto standard library for plot visualizations in Python.

[Seaborn][sns]
: A collection of routines and formats built on top of Matplotlib that provides a high-level interface for drawing attractive and informative statistical graphics. We mostly use it to provide a nice default plot appearance.

[np]: http://numpy.org/
[sp]: https://scipy.org/
[pd]: https://pandas.pydata.org/
[xr]: https://xarray.dev/
[mpl]: https://matplotlib.org/
[sns]: http://seaborn.pydata.org/

Using the file browser (the panel to the left of this notebook) open the file `data.txt` inside the `raw_data` folder. You should see data arrayed in two columns with "X" and "Y" in the first row (the column headers). Notice that the columns are separated by white space (a tab character, though it's difficult to tell the difference between a space and a tab in most editors). 

Now, click in the code cell below. Import the data from `data.txt` the `genfromtxt()` function in the NumPy library (just like in the Python Intro). Notice that we aren't specifying a delimiter this time---the `genfromtxt()` function does a pretty good job of guessing on its own. If we don't specify a `dtype`, `genfromtxt()` assumes we're dealing with numeric data and replaces non-numeric entries with `nan`, or "not a number", a special object in NumPy used for "numbers" that aren't valid numbers. 

:::{tip}
NumPy has a number of special [constants](https://numpy.org/doc/stable/reference/constants.html). The ones you're likely to encounter are `np.pi`, `np.e`, `np.nan`, and `np.inf`, representing $\pi$, $e$ (Euler's number, $\sim 2.718$), "Not a number", and $\infty$. 
:::

```python
data = np.genfromtxt("raw_data/data.txt", dtype='unicode')
```

Remember, before we can work with or plot this data, we need to separate the numeric data from the header and convert the numeric data to `float` values. We can quickly do that with the following two lines:

```python
header = data[0]
values = data[1:].astype(np.float64)
print(header)
print(values)
```

Now we can plot our data in a simple plot using Matplotlib's `pyplot`, which we imported as `plt` earlier. The main commands  used in this module are `plt.plot` (a single plt to be displayed at the end of your code cell), `plt.subplots` (for creating more complicated plots, arrays of plots, or plots with multiple axes), and `plt.fig` (for creating a plot you want to reference and update later). 

:::{note}
Remember, you can always look up the help file for a function using `help(function_name())` or, for brief references, by hitting <kbd>Shift</kbd> + <kbd>Tab</kbd> with your cursor inside the parentheses of the function while you're typing. 
:::

For now, let's start with `plt.plot()`. The general structure is `plt.plot(x1, y1, fmt1, x2, y2, fmt2, ...)`. `x1`, `x2`, `y1`, and `y2` indicate the _variable names_ for your $x$ and $y$ variables (these can be individual variables or slices of data arrays), while `fmt1` and `fmt2` are special format strings that Matplotlib uses to format data. For more detail, see the [help page for `plt.plot()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html), where the full list of short codes for `fmt` are listed. 

The code below will plot the values in our first column against the values in the second column using a dot for the point marker. If you leave out the `'.'` argument, `plt.plot()` will default to a straight line. 

```python
plt.plot(values[:,0],values[:,1], '.')
```

This plot is a little sparse… it's lacking simple content such as a title, axis labels, or a legend. Let's go back a couple of steps and see if we can use some of the structure in our input file to make up for some of that. 

Recall we mentioned that `genfromtxt()` can recognize the headers in a file; we're going to take advantage of that and the `names` argument to get our data into something slightly more structured. Import your data again using the following code:

```python
data = np.genfromtxt("raw_data/data.txt", dtype=np.float64, names=True)
data
```

Notice the data is now structured as pairs of numbers (indicated by parentheses around each row of data) and the `dtype` data at the bottom captured the column labels. Because of this structure, we can't slice our array the same way… previously, we had an $m \times n$ array of data, now we have a list of $m$ sets of coordinates. In other words, previously we could get the first entry in the second row using `data[1,0]`, and now we have to get the second row, then slice the first row (`data[1][0]`). _However,_ we can now address the data in each column separately by name, using `data['X']` and `data['Y']`.  

Try plotting the new data so the plot looks the same as before. 

### Polishing Plot Appearance

When creating more complex plots, once the basics are complete, it's good to remove any unnecessary "chart junk" to prepare your plot for publication. This includes things like background shading (Matplotlib is pretty good about this), grid lines (generally unnecessary unless you're trying to manually read data off the chart; we don't need this with the ubiquity of computers), and the border surrounding the full plot area (should only need axes to indicate the origin or provide an anchor for tick marks). If there is a legend (useful when there are multiple datasets present), it should usually be a part of the plot area, not pushed off to the side. Don't waste valuable data area on something like a legend than can be nestled in one of the empty parts of your plot. Finally, while color seems like a nice feature, plots shouldn't _rely_ on color to convey meaning. There's always a chance that readers won't have access to color printouts, or that the reader may be colorblind. To keep plots accessible, use symbols, line weights, and dashing to differentiate your datasets. The lesson to learn from this paragraph is that simplicity is key to conveying your message. The sole purpose of your figures is to help the reader understand the message. Anything that distracts from that is harmful to your cause.

This representation we created previously still very sparse… let's add some labels with the following code. 
:::{note}
When typing $\LaTeX$ code in text fields, the backslash character is frequently used for $\LaTeX$ commands. To make this process easier, you can use the format `r'$X$ vs. $Y$'`, called a "raw string",  to avoid needing to "escape" your backslashes (type two in a row to make sure the second one is processed). If you try to type code using a backslash (such as `$4 \pm 2$` to type 4±2) and you use a single backslash without indicating a raw string, the code will fail because `\p` isn't a valid escape character. If it _did_ happen to be valid, such as `\n`, you'd end up with a new line printed in the middle of your math. Just another reminder: computers do exactly what you _tell_ them to do, not what you _meant_ for them to do. 
:::

```python
plt.plot(data["X"], data["Y"], 'o')
plt.title(r"$X$ vs. $Y$")
plt.xlabel(r'$X$')
plt.ylabel(r'$Y$')
```

### Fitting Data

For the sake of familiarity, we'll introduce a [linear regression](https://en.wikipedia.org/wiki/Linear_regression) function from the stats module of SciPy. This will report values you're probably familiar with from linear fits using Excel (R-value, standard error of the slope and intercept). This isn't always the best way to characterize data, but it is simple and straightforward. Assuming your data is really linear, it works well enough. 

```python
from scipy import stats

lin_fit = stats.linregress(data["X"], data["Y"])
lin_fit
```

Now that we have some fit parameters, we can think about how to display this line with our data. Since all of our data in Python is represented by discrete points, we can't just say `plt.plot(f(x))`. Instead, we have to generate a set of $x$ and $y$ values that represent our function over the region of interest. The easiest way to start doing this is to create a set of evenly-spaced $x$ values, then do some basic calculations to transform them into $y$ values. NumPy has tools that make this procedure _very_ easy. 

The code below shows how to do this in the first two lines. The third line plots the results alongside our data. The next three lines are the same as before (title and axis labels), and the final block shows you how to create an annotation string. Notice we've combined a raw string with an f-string by using `fr'''string'''`. The triple quotes allow us to embed line breaks in the text without needing to explicitly add the `\n` linebreak escape code. 

```python
x_vals = np.arange(0,11)
y_vals = lin_fit.slope * x_vals + lin_fit.intercept

plt.plot(data["X"], data["Y"], "o", x_vals, y_vals, "-")

plt.title("$X$ vs. $Y$")
plt.xlabel('$X$')
plt.ylabel('$Y$')

plt.text(0,9, f'''The line of best fit is 
$y = {lin_fit.slope:1.4f}x + {lin_fit.intercept:1.4f}$
The R$^2$ value is {lin_fit.rvalue:1.4f}. ''')
```

While the fitting routine performed in the previous cell is sufficient for linearized data, it won't work for higher order polynomial fits or non-linear fits (_i.e.,_ exponential functions, logarithmic functions, or sigmoidal data)). The former requires the [Polynomial package from NumPy](https://numpy.org/doc/stable/reference/routines.polynomials.html), while the latter is best fitted using the [`curve_fit()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) routine from [SciPy's `optimize` package](https://docs.scipy.org/doc/scipy/reference/optimize.html). We'll investigate both of these later in the semester. 

### Anscombe's Quartet

Repeat the importing process (using the header line as names) with the file `anscombe.csv`. Name the new dataset `anscombe` and display it by calling it alone in the last line of your cell. You'll need to set the delimiter, so open the file in an editor and see how the columns are separated (the file extension should give you a clue). 

Now we'd like to calculate and print the linear fit parameters for each of the four datasets in `anscombe`. The columns should have names "x1", "y1", "x2", "y2", etc. For convenience, I've created a small loop for you to use on your data. Look over the loop and make sure you understand how it's working. 

```python
fits = []
for n in range(1,5):
    reg = sp.stats.linregress(anscombe[f'x{n}'], anscombe[f'y{n}'])
    fits.append([(reg[:2]), reg[-2:]])
    
# This portion makes my point regarding the data more clear, as you'll see when you run it
for item in fits:
    print(f"Slope = {item[0][0]:1.2f} ± {item[1][0]:1.2f}, Intercept = {item[0][1]:1.2f} ± {item[1][1]:1.2f}")
```

Clearly the data sets are different… you can easily see that from the number in the table. So what happened to all of our fits!? To learn more, plot each dataset in a separate plot. You can add a trendline if you like (they're all the same line…). Discuss your observations with your lab partner and come up with ways to deal with information and situations like this. 

The key takeaway from this exercise is that you should always plot your data, even if it's only a quick glance during your processing. This data was taken from an article written by F.J. Anscombe in 1973 arguing for the necessity of graphs in statistical analysis. 

## Plotting Spectra and Column Manipulation

Import the file `ftir_data.csv` in the cell below (define the delimiter and set `names=True`). Print out the first 5 rows of the data and note the names of the columns. You'll see that the column names aren't particularly nice… 

We could do a couple of things:

1. Deal with the messy name (lazy, will lead to issues later)
2. Edit the original file, the import the data (doesn't work well if the column headers were created by an external program or instrument)
3. Ignore the header using the `ignore_header` argument
4. Re-import the data and define the names with `names=["name1", "name2"]`
5. Edit the names directly

We're going to take the final route, just to show it can be done (the end result is the same as option 4). The column names are part of the `dtype` data for the array, accessible using dot notation with `ir_data.dtype.names`. This will output a list of column names and it can be reassigned on the fly. In the cell below, input the following

```python
ir_data.dtype.names = ["Wavenumber", "Transmittance"]
ir_data[:5]
```

Now that we've cleaned up our data a (tiny) bit, let's add some transformations to it. Due to some quirks of how NumPy arrays are stored, it's not trivial to add rows or columns to them. Instead, we're going to create a new 1-dimensional array with our transformed data and work on that for the moment. Our goal is to take this data in wavenumbers and transmittance and transform it to data in wavelength and absorbance units. To get you started, I've provided code to create the wavelength array. You'll need to do the unit analysis to figure out what units the final values are in (so you can appropriately label the axes of your plots). 

```python
wavelengths = 1 / (ir_data["Wavenumber"] / 10**(-2)) * 10**(6)
```
:::{note}
Python, like many other programming languages, uses the syntax `x**y` to denote "x raised to the y power" ($x^y$). This is because the caret character (^) is often used for other purposes.
:::

Create another variable that holds the calculated absorbance values. Once you've done this, create a pair of plots, the first plotting wavelength vs. absorbance, and the second plotting wavenumber vs. %T. Add labels and titles to your plots. 

## Importing and plotting data from multiple files

At this point, we'd like to consider how to handle and plot data from several files. In the next cell, we'll import data from a series of files taken using a Hitachi F-2700 fluorometer. The data is exported as a tab-separated table preceded by several lines of information ("metadata"). We'll break the code into several steps and walk through each part. 

First, we'll gather a list of files using a technique you've already seen: globbing. The files are in a folder called `fluor_data`, which is contained within the `raw_data` folder we've been using. Each file has a `.TXT` extension, so we'll set the file path to `'raw_data/fluor_data'`, then use the `glob('pattern')` method to match our desired pattern (`'*.TXT'`). 

```python
fluor_files = list(Path('raw_data/fluor_data').glob('*.TXT'))
```

Now that we have a list of files, we'll extract an identifying parameter from the filenames. We'll use the `string.split()` method to get out the percentage of one component in the mixture to identify our samples. We'll compile a list of these concentrations to label our imported files in a dictionary. We haven't worked with dictionaries yet, but they are, as named, a way of identifying various objects by a key in a single data structure. A simple one might look like:

```python
sample_dict = {'names':  ['Jason', 'Alec', 'Xin', 'Ncuti'],
               'majors': ['Math', 'History', 'English', 'Chemistry'],
               'age':    [21, 33, 18, 23]}

print(sample_dict['majors']) # ['Math', 'History', 'English', 'Chemistry']
```

The objects in a dictionary ("items)") are called "keys" and "values", and they can all be listed using the `items()`, `keys()`, and `values()` methods on a `dict()` object. We'll use our list of values as the keys for our dictionary. 

Finally, we're going to import all of our data into a dictionary of data, then plot a sorted version of that data. You'll add on the necessary labels and adjust the plot limits as appropriate at the end. The following block of code does a couple of things. First, we create an empty dictionary (either using `{}`, or `dict()`). The we loop over the files in our list to extract the concentration from the file name and import the contents into a dictionary item where the concentration is the name of the item key. Finally, we'll sort the list of keys and plot the data. 

:::{note}
Recall that `Path.stem` gives us just the name of a file, stripping off the file path and the file extension. Test the results for yourself by printing out the stem of one of the files in `fluor_files`. 
:::

```python
for file in fluor_files:
    conc = file.stem.split('%')[0]
    # While importing, we can skip the first 32 lines of the file. Open one to see why.
    data = np.genfromtxt(file, skip_header=32, names=['Wavelength', 'Concentration'])
    fluor_data[conc] = data['Concentration']
    # This next variable will get overwritten every loop, but that's ok, because all of
    # our data were taken with the same wavelength range. We could save the full data 
    # array, but that's a rather large duplication of data that we can reduce in this step
    wavelength = data['Wavelength']
    
# Finally, we sort the data (have to cast to integers for numeric sorting) so our data is 
# presented in a sensible manner. Dictionaries are unsorted by nature. 
sorted_keys = sorted([int(key) for key in fluor_data.keys()])
for key in sorted_keys:
    plt.plot(wavelength, fluor_data[str(key)], label=str(key))
    
plt.legend()
```